# Cargando los datos

El primer paso consistirá en cargar las librerías necesarias (pandas, seaborn, matplotlib y scikit-learn).

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Seguidamente, se procederá a cargar los datos a través del método de pandas "read_csv", renombrando la cabecera para hacerla más intuitiva.

In [2]:
labels = [
    'edad',
    'sexo',
    'dolor_pecho',
    'presion_arterial_reposo',
    'colesterol',
    'azúcar_ayunas',
    'electro_reposo',
    'frecuencia_máxima',
    'angina_por_ejercicio',
    'depresión_ST_por_ejercicio',
    'pendiente_ST_max_ejercicio',
    'número_vasos',
    'talio',
    'resultado'
]

In [3]:
df = pd.read_csv('heart.csv', header=0, names = labels)

In [4]:
df.head(15)

,edad,sexo,dolor_pecho,presion_arterial_reposo,colesterol,azúcar_ayunas,electro_reposo,frecuencia_máxima,angina_por_ejercicio,depresión_ST_por_ejercicio,pendiente_ST_max_ejercicio,número_vasos,talio,resultado
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   edad                        303 non-null    int64  
 1   sexo                        303 non-null    int64  
 2   dolor_pecho                 303 non-null    int64  
 3   presion_arterial_reposo     303 non-null    int64  
 4   colesterol                  303 non-null    int64  
 5   azúcar_ayunas               303 non-null    int64  
 6   electro_reposo              303 non-null    int64  
 7   frecuencia_máxima           303 non-null    int64  
 8   angina_por_ejercicio        303 non-null    int64  
 9   depresión_ST_por_ejercicio  303 non-null    float64
 10  pendiente_ST_max_ejercicio  303 non-null    int64  
 11  número_vasos                303 non-null    int64  
 12  talio                       303 non-null    int64  
 13  resultado                   303 non

Se comprueba que contamos con 303 entradas y que para ninguna de las categorías hay elementos nulos (NaN)

# Preprocesado de los datos

En relación a las variables categóricas, sería necesario realizar la transformación de las mismas a variables dummy. Del mismo modo, siempre que exista colinealidad directa entre dichas variables dummy, habría que eliminar las que resultaran recurrentes. Un ejemplo de esto sería el sexo, si un paciente es hombre, por definición no podrá ser mujer, entonces el género quedaría totalmente definido con una única columna en lugar de con dos al llevarse a cabo la transformación a valores dummy (0 ó 1).

In [6]:
df[df.duplicated()]

,edad,sexo,dolor_pecho,presion_arterial_reposo,colesterol,azúcar_ayunas,electro_reposo,frecuencia_máxima,angina_por_ejercicio,depresión_ST_por_ejercicio,pendiente_ST_max_ejercicio,número_vasos,talio,resultado
164,38,1,2,138,175,0,1,173,0,0.0,2,4,2,1


Revisando los posibles duplicados, comprobamos que contamos con una fila con información repetida. Procedemos a eliminarla:

In [7]:
df.drop_duplicates(keep='first',inplace=True)

Y comprobamos primeramente que, en efecto, tenemos una fila menos que inicialmente (302 filas) y que no contamos con duplicados:

In [8]:
df.shape

(302, 14)

In [9]:
df[df.duplicated()]

,edad,sexo,dolor_pecho,presion_arterial_reposo,colesterol,azúcar_ayunas,electro_reposo,frecuencia_máxima,angina_por_ejercicio,depresión_ST_por_ejercicio,pendiente_ST_max_ejercicio,número_vasos,talio,resultado


Ahora debemos dividir los datos en x (variables predictorias) e y (la variable dependiente, a predecir).

In [10]:
x = df.drop('resultado', axis=1)
x.head()

,edad,sexo,dolor_pecho,presion_arterial_reposo,colesterol,azúcar_ayunas,electro_reposo,frecuencia_máxima,angina_por_ejercicio,depresión_ST_por_ejercicio,pendiente_ST_max_ejercicio,número_vasos,talio
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [11]:
y = df[['resultado']]
y.head()

,resultado
0,1
1,1
2,1
3,1
4,1


Realizamos una lista con las variables categóricas (nos va a simplificar la transformación a variables dummy):

In [12]:
var_cat = ['sexo','dolor_pecho','azúcar_ayunas','electro_reposo','angina_por_ejercicio','pendiente_ST_max_ejercicio','número_vasos','talio']

In [13]:
x_dummy = pd.get_dummies(x, columns = var_cat)
x_dummy.head()

,edad,presion_arterial_reposo,colesterol,frecuencia_máxima,depresión_ST_por_ejercicio,sexo_0,sexo_1,dolor_pecho_0,dolor_pecho_1,dolor_pecho_2,...,pendiente_ST_max_ejercicio_2,número_vasos_0,número_vasos_1,número_vasos_2,número_vasos_3,número_vasos_4,talio_0,talio_1,talio_2,talio_3
0,63,145,233,150,2.3,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,37,130,250,187,3.5,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2,41,130,204,172,1.4,1,0,0,1,0,...,1,1,0,0,0,0,0,0,1,0
3,56,120,236,178,0.8,0,1,0,1,0,...,1,1,0,0,0,0,0,0,1,0
4,57,120,354,163,0.6,1,0,1,0,0,...,1,1,0,0,0,0,0,0,1,0


Pero como deseamos simplificar eliminando aquellas variables que presenten colinealidad directa, aplicamos el atributo drop_first = True, resultando:

In [14]:
x_dummy = pd.get_dummies(x, columns = var_cat, drop_first = True)
x_dummy.head()

,edad,presion_arterial_reposo,colesterol,frecuencia_máxima,depresión_ST_por_ejercicio,sexo_1,dolor_pecho_1,dolor_pecho_2,dolor_pecho_3,azúcar_ayunas_1,...,angina_por_ejercicio_1,pendiente_ST_max_ejercicio_1,pendiente_ST_max_ejercicio_2,número_vasos_1,número_vasos_2,número_vasos_3,número_vasos_4,talio_1,talio_2,talio_3
0,63,145,233,150,2.3,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
1,37,130,250,187,3.5,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,41,130,204,172,1.4,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,56,120,236,178,0.8,1,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,57,120,354,163,0.6,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0


Obsérvese como pasamos de contar con 30 columnas a 22.

A continuación, deberemos dividir los datos x e y en datos de entrenamiento (train) y de testeo (test) respectivamente. Para ello, aplicamos el siguiente método contenido en scikit-learn:

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y, test_size = 0.2)

Con estos pasos hemos preprocesado los datos, eliminando duplicados y transformando las variables categóricas a variables dummy interpretables por nuestros modelos predictivos. Adicionalmente, se han divido los datos en train y test.

# Entrenando el modelo predictivo

## Perceptrón multicapa (MLP)

En este apartado, estudiaremos las predicciones obtenidas a través del modelo de redes neuronales MLP para clasificación de la variable "resultado". Adicionalmente, se realizará una optimización de hiperparámetros para mejorar los resultados.

Instanciamos el modelo predictivo:

In [16]:
mlpc = MLPClassifier()

Definimos la matriz de hiperparámetros que queremos probar:

In [17]:
param_grid = {"hidden_layer_sizes": [(10), (5, 5), (10, 10), (10, 10, 10)],
             "activation": ["relu", "logistic", "tanh"],
             "solver": ["lbfgs", "sgd", "adam"]}

Se instancia el método GridSearchCV(), especificando que se ejecuta sobre el modelo o estimador MLP anteriormente declarado:

In [18]:
grid_classifier = GridSearchCV(estimator = mlpc,
                              param_grid = param_grid,
                              scoring = None,
                              cv = None,
                              n_jobs = -1)

Ajustamos el modelo aplicando los datos de entrenamiento y ejecutamos el comando %%time para comprobar los tiempos de ejecución:

In [19]:
%%time
grid_classifier.fit(x_train, y_train)

Wall time: 18.6 s


GridSearchCV(estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic', 'tanh'],
                         'hidden_layer_sizes': [10, (5, 5), (10, 10),
                                                (10, 10, 10)],
                         'solver': ['lbfgs', 'sgd', 'adam']})

Mostramos la mejor combinación de hiperparámetros:

In [20]:
grid_classifier.best_params_

{'activation': 'relu', 'hidden_layer_sizes': (10, 10), 'solver': 'lbfgs'}

Se instancia nuevamente el modelo MLP, teniendo en cuenta estos hiperparámetros óptimos. Sería posible recurrir a la misma variable ya definida, pero por claridad se ha optado por renombrarla otra vez a través de:

In [21]:
mlpc2 = MLPClassifier(activation = 'relu', hidden_layer_sizes = (10, 10), solver = 'lbfgs')

Ajustamos el nuevo modelo y realizamos las predicciones:

In [22]:
mlpc2.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), solver='lbfgs')

In [23]:
predicciones_mlpc2 = mlpc2.predict(x_test)

In [24]:
predicciones_mlpc2

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0], dtype=int64)

Y comprobamos el reporte de clasificación actualizado:

In [25]:
print(classification_report(y_test, predicciones_mlpc2))

              precision    recall  f1-score   support

           0       0.89      0.62      0.73        26
           1       0.77      0.94      0.85        35

    accuracy                           0.80        61
   macro avg       0.83      0.78      0.79        61
weighted avg       0.82      0.80      0.80        61



Con estos hierparámetros, alcanzamos un 89% de precisión para los casos en que hay menos probabilidad de ataque al corazón (valor de la variable "resultado" = 0), y con un 77% de acierto para los supuestos en los que hay alta probabilidad de sufrir un infarto (valor de la variable "resultado" = 1). Son resultados considerablemente buenos.

Con objeto de obtener mayor información, se realizar la representación de la matriz de confusión:

In [26]:
print(confusion_matrix(y_test, predicciones_mlpc2))

[[16 10]
 [ 2 33]]


Se comprueba que se obtienen 16 verdaderos positivos y 33 verdaderos negativos.